In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.config("spark.driver.memory", "6g").appName('chapter5').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 14:41:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/31 14:41:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Indetify Anamolous Network Traffic

In [6]:
data_without_header = spark.read.option("inferSchema", True).option("header", False).csv("/Users/quangtn/Desktop/01_work/01_job/02_ml/PySpark/chapter5/kddcup.data")

In [12]:
column_names = [  "duration", "protocol_type", "service", "flag",
  "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
  "hot", "num_failed_logins", "logged_in", "num_compromised",
  "root_shell", "su_attempted", "num_root", "num_file_creations",
  "num_shells", "num_access_files", "num_outbound_cmds",
  "is_host_login", "is_guest_login", "count", "srv_count",
  "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
  "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
  "dst_host_count", "dst_host_srv_count",
  "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate", "dst_host_srv_serror_rate",
  "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
  "label"]

In [13]:
data = data_without_header.toDF(*column_names)

In [14]:
from pyspark.sql.functions import col
data.select("label").groupBy("label").count().orderBy(col("count").desc()).show(25)

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
|            phf.|      4|
|           perl.|      3|
|            spy.|      2|
+----------------+-------+



In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline

numeric_only = data.drop("protocol_type", "service", "flag").cache()

23/12/31 14:59:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [17]:
assembler = VectorAssembler().setInputCols(numeric_only.columns[:-1]).setOutputCol("featureVector")

In [18]:
kmeans = KMeans().setPredictionCol("cluster").setFeaturesCol("featureVector")

In [19]:
pipeline = Pipeline().setStages([assembler, kmeans])
pipeline_model = pipeline.fit(numeric_only)

23/12/31 15:02:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/31 15:02:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [21]:
kmeans_model = pipeline_model.stages[1]

In [22]:
from pprint import pprint
pprint(kmeans_model.clusterCenters())

[array([4.83401949e+01, 1.83462155e+03, 8.26203190e+02, 5.71611720e-06,
       6.48779303e-04, 7.96173468e-06, 1.24376586e-02, 3.20510858e-05,
       1.43529049e-01, 8.08830584e-03, 6.81851124e-05, 3.67464677e-05,
       1.29349608e-02, 1.18874823e-03, 7.43095237e-05, 1.02114351e-03,
       0.00000000e+00, 4.08294086e-07, 8.35165553e-04, 3.34973508e+02,
       2.95267146e+02, 1.77970317e-01, 1.78036989e-01, 5.76648988e-02,
       5.77299094e-02, 7.89884132e-01, 2.11796106e-02, 2.82608101e-02,
       2.32981078e+02, 1.89214283e+02, 7.53713390e-01, 3.07109788e-02,
       6.05051931e-01, 6.46410789e-03, 1.78091184e-01, 1.77885898e-01,
       5.79276115e-02, 5.76592214e-02]),
 array([1.0999000e+04, 0.0000000e+00, 1.3099374e+09, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00

In [23]:
with_cluster = pipeline_model.transform(numeric_only)

In [24]:
with_cluster.select("cluster", "label").groupBy("cluster", "label").count().orderBy(col("cluster"), col("count").desc()).show(25)

+-------+----------------+-------+
|cluster|           label|  count|
+-------+----------------+-------+
|      0|          smurf.|2807886|
|      0|        neptune.|1072017|
|      0|         normal.| 972781|
|      0|          satan.|  15892|
|      0|        ipsweep.|  12481|
|      0|      portsweep.|  10412|
|      0|           nmap.|   2316|
|      0|           back.|   2203|
|      0|    warezclient.|   1020|
|      0|       teardrop.|    979|
|      0|            pod.|    264|
|      0|   guess_passwd.|     53|
|      0|buffer_overflow.|     30|
|      0|           land.|     21|
|      0|    warezmaster.|     20|
|      0|           imap.|     12|
|      0|        rootkit.|     10|
|      0|     loadmodule.|      9|
|      0|      ftp_write.|      8|
|      0|       multihop.|      7|
|      0|            phf.|      4|
|      0|           perl.|      3|
|      0|            spy.|      2|
|      1|      portsweep.|      1|
+-------+----------------+-------+



In [29]:
from pyspark.sql import DataFrame
from pyspark.ml.evaluation import ClusteringEvaluator
from random import randint

def clustering_score(input_data, k):
  input_numeric_only = input_data.drop("protocol_type", "service", "flag")
  assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
  kmeans = KMeans().setSeed(randint(100, 10000)).setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
  pipeline = Pipeline().setStages([assembler, kmeans])
  pipeline_model = pipeline.fit(input_numeric_only)

  evaluator = ClusteringEvaluator(predictionCol='cluster', featuresCol="featureVector")
  predictions = pipeline_model.transform(numeric_only)
  score = evaluator.evaluate(predictions)
  return score

In [30]:
for k in list(range(20, 40, 20)):
  print(clustering_score(numeric_only, k))

0.9996553386887673


In [35]:
def clustering_score_1(input_data, k):
    input_numeric_only = input_data.drop("protocol_type", "service", "flag")
    assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
    kmeans = KMeans().setSeed(randint(100, 100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("featureVector")
    pipeline = Pipeline().setStages([assembler, kmeans])
    pipeline_model = pipeline.fit(input_numeric_only)

    evaluator = ClusteringEvaluator(predictionCol='cluster', featuresCol="featureVector")
    predictions = pipeline_model.transform(numeric_only)
    score = evaluator.evaluate(predictions)

    return score

In [36]:
for k in list(range(20,41,20)):
    print(k, clustering_score_1(numeric_only, k))

20 0.9997630974591062


40 0.9997063047834809


In [37]:
from pyspark.ml.feature import StandardScaler

In [38]:
def clustering_score_2(input_data, k):
    input_numeric_only = input_data.drop("protocol_type", "service", "flag")
    assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
    scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
    kmeans = KMeans().setSeed(randint(100, 10000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("scaledFeatureVector")
    pipeline = Pipeline().setStages([assembler, scaler, kmeans])
    pipeline_model = pipeline.fit(input_numeric_only)

    evaluator = ClusteringEvaluator(predictionCol="cluster", featuresCol="scaledFeatureVector")
    predictions = pipeline_model.transform(numeric_only)
    score = evaluator.evaluate(predictions)
    return score

In [39]:
for k in list(range(60,81,20)):
    print(k, clustering_score_2(numeric_only,k))

60 0.8414967421804316


80 0.9018280253834207


In [43]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

def one_hot_pipeline(input_col):
    indexer = StringIndexer().setInputCol(input_col).setOutputCol(input_col + "-_indexed")
    encoder = OneHotEncoder().setInputCol(input_col).setOutputCol(input_col + "_vec")
    pipeline = Pipeline().setStages([indexer, encoder])
    return pipeline, input_col +"_vec"

In [52]:
def clustering_score_3(input_data, k):
    proto_type_pipeline, proto_type_vec_col = one_hot_pipeline("protocol_type")
    service_pipeline, service_vec_col = one_hot_pipeline("service")
    flag_pipeline, flag_vec_col = one_hot_pipeline("flag")

    assemble_cols = set(input_data.columns) - {"label", "protocol_type", "service" , "flag"} | \
                {proto_type_vec_col, service_vec_col, flag_vec_col}

    assembler = VectorAssembler().setInputCols(list(assemble_cols)).setOutputCol("featureVector")
    scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
    kmeans = KMeans().setSeed(randint(100, 100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("scaledFeatureVector")
    pipeline = Pipeline().setStages([proto_type_pipeline, service_pipeline, flag_pipeline, assembler, scaler, kmeans])
    pipeline_model = pipeline.fit(input_data)

    kmeans_model = pipeline_model.stages[-1]
    training_cost = kmeas_model.summary.trainingCost
    return training_cost

In [55]:
#for k in list(range(60, 81, 20)):
#    print(k, clustering_score_3(data, k))

In [56]:
from math import log

In [57]:
def entropy(counts):
    values = [c for c in counts if (c>0)]
    n = sum(values)
    p = [v/n for v in values]
    return sum([-1*(p_v) * log(p_v) for p_v in p])

In [58]:
from pyspark.sql import functions as fun
from pyspark.sql import Window

In [59]:
cluster_label = pipeline_model.transform(data).select("cluster", "label")

In [60]:
df = cluster_label.groupBy("cluster", "label").count().orderBy("cluster")

In [64]:
w = Window.partitionBy("cluster")
p_col = df['count']/fun.sum(df['count']).over(w)
with_p_col = df.withColumn("p_col", p_col)

result = with_p_col.groupBy("cluster").agg(fun.sum(col("p_col")*fun.log2(col("p_col"))).alias("entropy"), fun.sum(col("count")).alias("cluster_size"))

In [65]:
result = result.withColumn('weightedClusterEntropy', col('entropy')*col('cluster_size'))

In [66]:
weighted_cluster_entropy_avg = result.agg(fun.sum(col('weightedClusterEntropy'))).collect()

In [67]:
weighted_cluster_entropy_avg[0][0]/data.count()

-1.4873454858776902

In [71]:
def fit_pipeline_4(data, k):
    (proto_type_pipeline, proto_type_vec_col) = one_hot_pipeline("protocol_type")
    (service_pipeline, service_vec_col) = one_hot_pipeline("service")
    (flag_pipeline, flag_vec_col) = one_hot_pipeline("flag")

    assemble_cols = set(data.columns) - {"label", "protocol_type", "service", "flag"} | {proto_type_vec_col, service_vec_col, flag_vec_col}
    assembler = VectorAssembler(inputCols=list(assemble_cols), outputCol="featureVector")

    scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)

    kmeans = KMeans(seed=randint(100, 10000), k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-5)
    pipeline = Pipeline(stages=[proto_type_pipeline, service_pipeline, flag_pipeline, assembler, scaler, kmeans])
    return pipeline.fit(data)

In [73]:
pipeline_model = fit_pipeline_4(data, 180)
count_by_cluster_label = pipeline_model.transform(data).select("cluster", "label").groupBy("cluster", "label").count().orderBy("cluster", "label")
count_by_cluster_label.show()

In [74]:
import numpy as np

from pyspark.ml.linalg import Vector, Vectors
from pyspark.sql.functions import udf

In [75]:
k_means_model = pipeline_model.stages[-1]
centroids = k_means_model.clusterCenters

In [76]:
clustered = pipeline_model.transform(data)

In [77]:
def dist_func(cluster, vec):
    return float(np.linalg.norm(centroids[cluster] - vec))

In [79]:
threshold = clustered.select("cluster", "scaledFeatureVector").withColumn("dist_value", dist(col("cluster"), col("scaledFeatureVector"))).orderBy(col("dist_value").desc()).take(100)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `scaledFeatureVector` cannot be resolved. Did you mean one of the following? [`featureVector`, `cluster`, `same_srv_rate`, `urgent`, `count`].;
'Project [cluster#40343, 'scaledFeatureVector]
+- Project [duration#101, protocol_type#102, service#103, flag#104, src_bytes#105, dst_bytes#106, land#107, wrong_fragment#108, urgent#109, hot#110, num_failed_logins#111, logged_in#112, num_compromised#113, root_shell#114, su_attempted#115, num_root#116, num_file_creations#117, num_shells#118, num_access_files#119, num_outbound_cmds#120, is_host_login#121, is_guest_login#122, count#123, srv_count#124, ... 20 more fields]
   +- Project [duration#101, protocol_type#102, service#103, flag#104, src_bytes#105, dst_bytes#106, land#107, wrong_fragment#108, urgent#109, hot#110, num_failed_logins#111, logged_in#112, num_compromised#113, root_shell#114, su_attempted#115, num_root#116, num_file_creations#117, num_